In [14]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions

options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)

In [13]:
import apache_beam as beam
from apache_beam.runners import DataflowRunner
from apache_beam.io.kafka import ReadFromKafka
from apache_beam.io.kafka import WriteToKafka
import json

bootstrap_servers = '34.28.118.32:9094'
topic = 'my-topic'
topic2 = 'my-second-topic'
write_topic = 'my-fifth-topic'


p = beam.Pipeline(options=options)
stage1 = p| ReadFromKafka(
            consumer_config={'bootstrap.servers': bootstrap_servers},
            topics=[topic])
stage3 = stage1|beam.Map(lambda x: x[1].decode('utf8'))
stage4 = stage3|beam.Map(lambda x: json.loads(x))
stage5 = stage4|beam.WindowInto(beam.window.FixedWindows(20))
stage6 = stage5| "Write to Bigquery">>beam.io.Write(
                                               beam.io.WriteToBigQuery(
                                                                        'my-table-kafka',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='v1:FLOAT,v2:FLOAT,v3:FLOAT,v4:FLOAT,id:INTEGER,Time:INTEGER',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       )

pipeline_result = DataflowRunner().run_pipeline(p, options=options)


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip


In [15]:
import apache_beam as beam
from apache_beam.runners import DataflowRunner
from apache_beam.io.kafka import ReadFromKafka
from apache_beam.io.kafka import WriteToKafka
import json

bootstrap_servers = '34.28.118.32:9094'
topic = 'my-topic'
topic2 = 'my-second-topic'
write_topic = 'my-fifth-topic'


p = beam.Pipeline(options=options)
stage1 = p| ReadFromKafka(
            consumer_config={'bootstrap.servers': bootstrap_servers},
            topics=[topic])
stage3 = stage1|beam.Map(lambda x: x[1].decode('utf8'))
stage4 = stage3|beam.Map(lambda x: json.loads(x))
stage5 = stage4|beam.WindowInto(beam.window.FixedWindows(20))
stage6 = stage5| "Write to Bigquery">>beam.io.Write(
                                               beam.io.WriteToBigQuery(
                                                                        'my-table-customer-records',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='id:STRING,name:STRING,Time:INTEGER',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       )

pipeline_result = DataflowRunner().run_pipeline(p, options=options)


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip
